# 🌍 Chapter 5: 3D Multimodal Viewer

In this chapter, we will build a comprehensive 3D viewer that can handle disparate data sources: Point Clouds and 3D Meshes. We will also perform advanced spatial analysis like calculating built coverage and finding points of interest.

**Objectives:**
1.  **Multimodal Loading**: Load and visualize both `.xyz` point clouds and `.obj` meshes.
2.  **Pre-processing**: Apply sampling and outlier removal techniques.
3.  **Visualization**: Combine different 3D entities in a single scene.
4.  **Spatial Analysis**: Determine urban density and query specific areas.

## 1. Setup and Loading
We start by importing the necessary libraries and defining our dataset paths.

In [ ]:
import numpy as np
import pandas as pd
import open3d as o3d
from shapely.geometry import Polygon

print(f"Open3D Version: {o3d.__version__}")

In [ ]:
# Define paths to datasets
data_folder = "../DATA/"
pc_dataset = "30HZ1_18_sampled.xyz"
mesh_dataset = "NL.IMBAG.Pand.0637100000139735.obj"

# Load Point Cloud using Pandas for flexibility
try:
    pcd_df = pd.read_csv(data_folder + pc_dataset, delimiter=";")
    print("Point Cloud Columns:", pcd_df.columns)
except FileNotFoundError:
    print(f"⚠️ Error: Could not find {pc_dataset}. Make sure it is in the DATA folder.")

# Load Mesh
try:
    mesh = o3d.io.read_triangle_mesh(data_folder + mesh_dataset)
    mesh.paint_uniform_color([0.9, 0.9, 0.9])  # Paint mesh light grey
    mesh.compute_vertex_normals()
    print(f"Mesh loaded: {len(mesh.triangles)} triangles.")
except Exception as e:
    print(f"⚠️ Error loading mesh: {e}")

## 2. Converting to Open3D

We need to convert our Pandas dataframe into an Open3D PointCloud object to utilize the visualization tools.

In [ ]:
if 'pcd_df' in locals():
    # Initialize Open3D PointCloud
    pcd_o3d = o3d.geometry.PointCloud()
    
    # Set Geometry (XYZ)
    pcd_o3d.points = o3d.utility.Vector3dVector(np.array(pcd_df[['X', 'Y', 'Z']]))
    
    # Set Color (RGB) - Normalize 0-255 to 0-1
    pcd_o3d.colors = o3d.utility.Vector3dVector(np.array(pcd_df[['R', 'G', 'B']]) / 255.0)
    
    print(f"Converted {len(pcd_o3d.points)} points to Open3D format.")

    # Center the data for better visualization
    pcd_center = pcd_o3d.get_center()
    pcd_o3d.translate(-pcd_center)
    # We must also translate the mesh to match the centralized point cloud
    mesh.translate(-pcd_center)

## 3. Data Pre-Processing

Large datasets can be unwieldy. We use **Downsampling** to reduce the point count while maintaining the overall structure, and **Statistical Outlier Removal** to clean up noise.

In [ ]:
if 'pcd_o3d' in locals():
    # 1. Random Downsampling
    # Keep only 20% of points
    sampled_pcd = pcd_o3d.random_down_sample(0.2)
    print(f"Random Sampling: {len(sampled_pcd.points)} points remaining.")

    # 2. Voxel Downsampling (More uniform)
    voxel_size = 0.05
    pcd_downsampled = pcd_o3d.voxel_down_sample(voxel_size=voxel_size)
    print(f"Voxel Sampling: {len(pcd_downsampled.points)} points remaining.")

    # 3. Statistical Outlier Removal
    # Removes points that are further away from their neighbors compared to the average
    nb_neighbors = 16
    std_ratio = 2.0  # Lower threshold = more aggressive filtering
    
    cl, ind = pcd_downsampled.remove_statistical_outlier(nb_neighbors=nb_neighbors, std_ratio=std_ratio)
    pcd_clean = pcd_downsampled.select_by_index(ind)
    pcd_outliers = pcd_downsampled.select_by_index(ind, invert=True)
    
    # Visualize Outliers in Red
    pcd_outliers.paint_uniform_color([1, 0, 0])
    print(f"Outlier Removal: Removed {len(pcd_outliers.points)} noise points.")
    
    o3d.visualization.draw_geometries([pcd_clean, pcd_outliers, mesh], window_name="Outlier Visualization")

## 4. Advanced Visualization

We can color-code the point cloud based on its classification classes (Ground, Building, Water, etc.).

*   **1**: Unclassified
*   **2**: Ground
*   **6**: Building
*   **9**: Water
*   **26**: Other

In [ ]:
if 'pcd_df' in locals():
    classes = pcd_df['Classification'].unique()
    print(f"Classes found: {classes}")

    # Color mapping
    colors = np.zeros((len(pcd_df), 3))
    colors[pcd_df['Classification'] == 1] = [0.6, 0.8, 0.5]  # Unclassified (Greenish)
    colors[pcd_df['Classification'] == 2] = [0.8, 0.7, 0.5]  # Ground (Brown)
    colors[pcd_df['Classification'] == 6] = [0.9, 0.4, 0.4]  # Building (Red)
    colors[pcd_df['Classification'] == 9] = [0.5, 0.8, 0.9]  # Water (Blue)
    colors[pcd_df['Classification'] == 26] = [0.7, 0.7, 0.7] # Other (Grey)
    
    # Apply colors to a fresh point cloud object so we don't mess up the downsampled one for later
    pcd_classified = o3d.geometry.PointCloud(pcd_o3d)
    pcd_classified.colors = o3d.utility.Vector3dVector(colors)
    
    o3d.visualization.draw_geometries([pcd_classified, mesh], window_name="Classified Point Cloud")

## 5. Spatial Analysis

### 5.1 POI Query
Find points within a certain radius of a specific location (Point of Interest). We use a **KDTree** for fast neighbour search.

In [ ]:
# Define POI as the center of the mesh (the building)
poi = mesh.get_center()
search_radius = 50.0  # meters

# Build KDTree
pcd_tree = o3d.geometry.KDTreeFlann(pcd_o3d)

# Search
[k, idx, _] = pcd_tree.search_radius_vector_3d(poi, search_radius)

# Select points
pcd_poi = pcd_o3d.select_by_index(idx)
print(f"Found {len(pcd_poi.points)} points within {search_radius}m of the building.")

o3d.visualization.draw_geometries([pcd_poi, mesh], window_name="POI Selection")

### 5.2 Volumetric Analysis (Built Coverage)
We can voxelize the selected area to determine how much of the space is occupied by structures.

In [ ]:
# Create Voxel Grid from the POI selection
voxel_size = 2.0
voxel_grid = o3d.geometry.VoxelGrid.create_from_point_cloud(pcd_poi, voxel_size=voxel_size)

o3d.visualization.draw_geometries([voxel_grid], window_name="Voxel Analysis")

### Note on Polygon Analysis
The original notebook contained polygon area calculation. This requires specific corner points. For general use, refer to `shapely` documentation for polygon operations once you have extracted corner coordinates.